In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117
!pip install pytorch_lightning==1.9.4
!pip install pytorch_forecasting==0.10.3
!pip install gluonts

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 998.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 41.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.2+cu118
    Uninstalling torchvision-0.15.2+cu118:
      Successfully uninstalled torchvision-0.15.2+cu118
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.2+cu118
    Uninstalling torchaudio-2.0.2+cu118:
      Successfully uninstalled torchaudio-2.0.2+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the so

In [3]:
import pandas as pd
import numpy as np
from gluonts.dataset.pandas import PandasDataset
from gluonts.torch.model.deepar import DeepAREstimator
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [4]:
def SMAPE(true, pred):
    return np.mean((np.abs(true-pred))/(np.abs(true) + np.abs(pred))) * 100

In [5]:
def wmape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()

In [6]:
path = '/content/drive/MyDrive/DACON/lg3/'

In [7]:
df = pd.read_csv(path+'df_all2.csv')
test = pd.read_csv(path+'test_all2.csv')
sub = pd.read_csv(path + 'new_sub.csv')

In [8]:
sub.drop(columns=['ansswer'], inplace=True)
sub

,product_date_time,answer
0,00001-00001_C001-0002_C002-0007_C003-0038_0000...,0
1,00001-00001_C001-0002_C002-0007_C003-0038_0000...,0
2,00001-00001_C001-0002_C002-0007_C003-0038_0000...,0
3,00001-00001_C001-0002_C002-0007_C003-0038_0000...,0
4,00001-00001_C001-0002_C002-0007_C003-0038_0000...,0
...,...,...
333685,03799-00010_C001-0002_C002-0004_C003-0020_0379...,0
333686,03799-00010_C001-0002_C002-0004_C003-0020_0379...,0
333687,03799-00010_C001-0002_C002-0004_C003-0020_0379...,0
333688,03799-00010_C001-0002_C002-0004_C003-0020_0379...,0


In [10]:
df['datetime'] = pd.to_datetime(df.datetime)
test['datetime'] = pd.to_datetime(test.datetime)

In [ ]:
# cutoff_datetime = pd.to_datetime('2022-08-18 00:00:00')
# train = df.loc[df['datetime'] < cutoff_datetime]
# valid = df.loc[df['datetime'] >= cutoff_datetime]

In [11]:
train_ds = PandasDataset.from_long_dataframe(df, target='target', item_id='product',
                                       timestamp='datetime', freq='D')

In [ ]:
context_length_list = [21*i for i in range(1,25)]
for length in context_length_list:
    estimator = DeepAREstimator(freq='D', prediction_length=21, num_layers=5, context_length=length, trainer_kwargs={'accelerator': 'gpu', 'max_epochs':100})
    predictor = estimator.train(train_ds, num_workers=2)
    pred = list(predictor.predict(train_ds))

    all_preds = []
    for i in tqdm(range(len(pred))):
        value = pred[i].samples.mean(axis=0)
        for j in range(len(value)):
            all_preds.append(value[j])

    sub['answer'] = all_preds
    sub.to_csv(path + f'deepar/deepar_{length}.csv', index=False)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type        | Params | In sizes | Out sizes   
----------------------------------------------------------------
0 | model | DeepARModel | 65.2 K | ?        | [1, 100, 21]
----------------------------------------------------------------
65.2 K    Trainable params
0         Non-trainable params
65.2 K    Total param

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 50: 'train_loss' reached -8.74171 (best -8.74171), saving model to '/content/lightning_logs/version_0/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 1, global step 100: 'train_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 2, global step 150: 'train_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 200: 'train_loss' reached -8.92668 (best -8.92668), saving model to '/content/lightning_logs/version_0/checkpoints/epoch=3-step=200.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 4, global step 250: 'train_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 5, global step 300: 'train_loss' reached -10.24720 (best -10.24720), saving model to '/content/lightning_logs/version_0/checkpoints/epoch=5-step=300.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 6, global step 350: '

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 50: 'train_loss' reached -6.35562 (best -6.35562), saving model to '/content/lightning_logs/version_1/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 1, global step 100: 'train_loss' reached -7.36484 (best -7.36484), saving model to '/content/lightning_logs/version_1/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 2, global step 150: 'train_loss' reached -7.83647 (best -7.83647), saving model to '/content/lightning_logs/version_1/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 200: 'train_loss' reached -8.32849 (best -8.32849), saving model to '/content/lightning_logs/version_1/checkpoints/epoch=3-step=200.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 4, global step 250: 'train_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 5, global 

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 50: 'train_loss' reached -1.32285 (best -1.32285), saving model to '/content/lightning_logs/version_14/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 1, global step 100: 'train_loss' reached -1.77082 (best -1.77082), saving model to '/content/lightning_logs/version_14/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 2, global step 150: 'train_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 200: 'train_loss' reached -2.12754 (best -2.12754), saving model to '/content/lightning_logs/version_14/checkpoints/epoch=3-step=200.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 4, global step 250: 'train_loss' reached -2.28157 (best -2.28157), saving model to '/content/lightning_logs/version_14/checkpoints/epoch=4-step=250.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 5, glo

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 50: 'train_loss' reached -0.92094 (best -0.92094), saving model to '/content/lightning_logs/version_15/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 1, global step 100: 'train_loss' reached -1.38777 (best -1.38777), saving model to '/content/lightning_logs/version_15/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 2, global step 150: 'train_loss' reached -1.67012 (best -1.67012), saving model to '/content/lightning_logs/version_15/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 200: 'train_loss' reached -2.41631 (best -2.41631), saving model to '/content/lightning_logs/version_15/checkpoints/epoch=3-step=200.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 4, global step 250: 'train_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 5, glo

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 50: 'train_loss' reached -0.90631 (best -0.90631), saving model to '/content/lightning_logs/version_16/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 1, global step 100: 'train_loss' reached -1.18513 (best -1.18513), saving model to '/content/lightning_logs/version_16/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 2, global step 150: 'train_loss' reached -2.04092 (best -2.04092), saving model to '/content/lightning_logs/version_16/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 200: 'train_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 4, global step 250: 'train_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 5, global step 300: 'train_loss' reached -2.12135 (best -2.12135), saving model to '/content/lightning_logs/version_16/

In [ ]:
estimator = DeepAREstimator(freq='h', prediction_length=7*24, num_layers=5, context_length=7*24*11, trainer_kwargs={'accelerator': 'gpu', 'max_epochs':30})

predictor = estimator.train(train_ds, num_workers=2)

pred = list(predictor.predict(train_ds))

all_preds = list()
for item in pred:
    family = int(item.item_id)
    p = item.samples.mean(axis=0)
    p10 = np.percentile(item.samples, 10, axis=0)
    p90 = np.percentile(item.samples, 90, axis=0)
    dates = pd.date_range(start=item.start_date.to_timestamp(), periods=len(p), freq='h')
    family_pred = pd.DataFrame({'datetime': dates, 'num': family, 'pred': p, 'p10': p10, 'p90': p90})
    all_preds += [family_pred]
all_preds = pd.concat(all_preds, ignore_index=True)
all_preds = all_preds.merge(valid, on=['datetime', 'num'], how='left')

SMAPE(all_preds['target'], all_preds['pred'])

import matplotlib.pyplot as plt
fig, ax = plt.subplots(3,2, figsize=(1280/96, 720/96), dpi=96)
ax = ax.flatten()
for ax_ ,family in enumerate(all_preds['num'].unique()[:6]):
    p_ = all_preds.loc[all_preds['num'] == family]
    p_.plot(x='datetime', y='target', ax=ax[ax_], label='Target')
    p_.plot(x='datetime', y='pred', ax=ax[ax_], label='Forecast')
    ax[ax_].fill_between(p_['datetime'].values, p_['p10'], p_['p90'], alpha=0.2, color='orange')
    ax[ax_].set_title(family)
    ax[ax_].legend()
    ax[ax_].set_xlabel('Date')
    ax[ax_].set_ylabel('Target')
fig.tight_layout()

In [ ]:
# pred = list(predictor.predict(train_ds))

# all_preds = []
# for i in tqdm(range(len(pred))):
#     value = pred[i].samples.mean(axis=0)
#     for j in range(len(value)):
#         all_preds.append(value[j])

# test['answer'] = all_preds

# product_unique = test['product'].unique()

# count = 0
# for product in tqdm(product_unique):
#     answer_list = []
#     answer_list.append(count)
#     df = test[test['product'] == product]
#     df.reset_index(drop=True, inplace=True)
#     for i in range(len(df)):
#         value = np.round(df['answer'][i])
#         if value < 0:
#             value = 0
#         answer_list.append(value)
#     sub.loc[count] = answer_list
#     count += 1

# sub.to_csv(path + 'deepar.csv', index=False)

In [ ]:
# sub.loc[224]

In [ ]:
# sub.loc[485]